In [30]:
# coding: utf-8
import tensorflow as tf
import os
import numpy as np
from copy import deepcopy
import sys

from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.optimizers import Adam
from keras import backend as K
from keras.callbacks import EarlyStopping

In [31]:
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)

file_index = 1
noise_level = 0.5
clean_data_size = 50
seed = 10 * file_index
additional_data_size = 2000
learning_rate = 0.0003
iteration_num = 20

np.random.seed(seed)
tf.set_random_seed(seed)

x_train, y_train, x_test, y_test, x_clean, y_clean = load_data(clean_data_size)
y_train_orig = deepcopy(y_train)
y_train = generate_noise_labels(y_train, noise_level)

In [32]:
# load data from CIFAR10
def load_data(clean_data_size):
    cifar10 = tf.keras.datasets.cifar10
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)
    x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)

    # transform labels to one-hot vectors
    y_train = tf.contrib.keras.utils.to_categorical(y_train, 10)
    y_test = tf.contrib.keras.utils.to_categorical(y_test, 10)

    clean_index = []
    for label in range(10):
        positive_index = list(np.where(y_train[:, label] == 1)[0])
        clean_index = np.append(clean_index, np.random.choice(positive_index, clean_data_size, replace=False)).astype(
            int)

    x_clean = x_train[clean_index]
    y_clean = y_train[clean_index]
    x_train = np.delete(x_train, clean_index, axis=0)
    y_train = np.delete(y_train, clean_index, axis=0)
    return x_train, y_train, x_test, y_test, x_clean, y_clean


def generate_noise_labels(y_train, noise_level):
    num_noise = int(noise_level * y_train.shape[0])
    noise_index = np.random.choice(y_train.shape[0], num_noise, replace=False)
    label_slice = np.argmax(y_train[noise_index], axis=1)
    new_label = np.random.randint(low=0, high=10, size=num_noise)
    while sum(label_slice == new_label) > 0:
        n = sum(label_slice == new_label)
        new_label[label_slice == new_label] = np.random.randint(low=0, high=10, size=n)
    y_train[noise_index] = tf.contrib.keras.utils.to_categorical(new_label, 10)
    return y_train


def create_model(architecture, num_classes, learning_rate, dropout=0.5):
    model = Sequential()
    for layer_index in range(len(architecture)):
        layer = architecture[layer_index]
        if len(layer) == 3:
            if layer_index == 0:
                model.add(Conv2D(layer[0], kernel_size=(layer[1], layer[2]), input_shape=(32, 32, 3),
                                 kernel_initializer='glorot_normal', activation='relu', padding='same'))
            else:
                model.add(Conv2D(layer[0], kernel_size=(layer[1], layer[2]), kernel_initializer='glorot_normal',
                                 activation='relu', padding='same'))
            if layer_index < 3:
                model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        elif len(layer) == 1:
            if len(architecture[layer_index - 1]) == 3:
                model.add(Flatten())
            model.add(Dense(layer[0], activation='relu', kernel_initializer='glorot_normal'))
        else:
            print('Invalid architecture /(ㄒoㄒ)/~~')
    model.add(Dropout(dropout))
    if num_classes > 2:
        model.add(Dense(num_classes))
        model.add(Activation('softmax'))
        adam = Adam(lr=learning_rate)
        model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)
    elif num_classes == 2:
        model.add(Dense(1))
        model.add(Activation('sigmoid'))
        adam = Adam(lr=learning_rate)
        model.compile(loss='mean_squared_error', metrics=['accuracy'], optimizer=adam)
    return model

In [33]:
def select_additional_data(x, binary_classifier_list, additional_data_size):    
    n = len(binary_classifier_list)
    y = np.zeros((len(x), n))
    
    for index in range(n):
        model = binary_classifier_list[index]
        y_pred = model.predict(x).reshape(-1,)
        y[:, index] = y_pred>0.5 
    
    y = np.sum(y, axis=1)
    
    add_index = np.where(y > 90)[0]
    if len(add_index) > additional_data_size:
        add_index = np.argsort(-y, axis=0)[0:additional_data_size].reshape(-1)
    return add_index

In [ ]:
architecture = [[32, 5, 5], [32, 5, 5], [32, 5, 5], [256]]
additional_data_index = [[] for i in range(10)]

for iteration in range(iteration_num):
    for label in range(10):
        binary_classifier_list = []
        
        # train 100 binary classifiers for one label
        for index in range(100):
            print('iteration', iteration, 'label', label, 'index', index)
            model = create_model(architecture, num_classes=2, learning_rate=learning_rate)
            positive_index = list(np.where(y_clean[:, label] == 1)[0])
            x = x_clean[positive_index]
            
            if len(additional_data_index[label]) < 950:                
                add_positive_index = additional_data_index[label]
            else:
                add_positive_index = np.random.choice(additional_data_index[label], 950, replace=False)
            x = np.concatenate((x, x_train[add_positive_index]), axis=0)
                    
            n_p = len(x)
            n_n_clean = min(450, n_p//2)
            n_n_noisy = n_p - n_n_clean
            negative_index_clean = list(np.where(y_clean[:, label] != 1)[0])
            negative_index_clean = np.random.choice(negative_index_clean, n_n_clean, replace=False)
            x = np.concatenate((x, x_clean[negative_index_clean]), axis=0)
            negative_index_noisy = set(np.arange(len(x_train))) - set(additional_data_index[label])
            negative_index_noisy = np.random.choice(list(negative_index_noisy), n_n_noisy, replace=False)
            x = np.concatenate((x, x_train[negative_index_noisy]), axis=0)
            
            y = [1] * n_p + [0] * n_p
            early_stopping = EarlyStopping(monitor='loss', patience=5)
            model.fit(x, y, batch_size=32, epochs=50, shuffle=True, callbacks=[early_stopping])
            binary_classifier_list.append(model)          
        additional_data_index[label] = select_additional_data(x_train, binary_classifier_list, additional_data_size)
        
    # estimate additional clean data
    precision_additional_data = []
    number_additional_data = []
    for label in range(10):
        index = additional_data_index[label]
        true_positive_index = list(np.where(y_train_orig[:, label] == 1)[0])
        TP = len(list(set(index) & set(true_positive_index)))
        if len(index) == 0:
            precision_additional_data.append(0)
        else:
            precision_additional_data.append(TP / len(index))
        number_additional_data.append(len(index))
    print(precision_additional_data)
    print(number_additional_data)

iteration 0 label 0 index 0
Epoch 1/50
100/100 [==============================] - 5s 49ms/step - loss: 0.2561 - acc: 0.4700
Epoch 2/50
100/100 [==============================] - 0s 251us/step - loss: 0.2449 - acc: 0.5200
Epoch 3/50
100/100 [==============================] - 0s 237us/step - loss: 0.2458 - acc: 0.5400
Epoch 4/50
100/100 [==============================] - 0s 847us/step - loss: 0.2452 - acc: 0.6000
Epoch 5/50
100/100 [==============================] - 0s 943us/step - loss: 0.2499 - acc: 0.5400
Epoch 6/50
100/100 [==============================] - 0s 997us/step - loss: 0.2399 - acc: 0.5200
Epoch 7/50
100/100 [==============================] - 0s 818us/step - loss: 0.2457 - acc: 0.5800
Epoch 8/50
100/100 [==============================] - 0s 985us/step - loss: 0.2454 - acc: 0.6200
Epoch 9/50
100/100 [==============================] - 0s 896us/step - loss: 0.2394 - acc: 0.5600
Epoch 10/50
100/100 [==============================] - 0s 251us/step - loss: 0.2358 - acc: 0.6900
Ep

100/100 [==============================] - 0s 264us/step - loss: 0.2518 - acc: 0.4900
Epoch 5/50
100/100 [==============================] - 0s 271us/step - loss: 0.2412 - acc: 0.6800
Epoch 6/50
100/100 [==============================] - 0s 263us/step - loss: 0.2424 - acc: 0.6400
Epoch 7/50
100/100 [==============================] - 0s 226us/step - loss: 0.2428 - acc: 0.6100
Epoch 8/50
100/100 [==============================] - 0s 242us/step - loss: 0.2379 - acc: 0.6400
Epoch 9/50
100/100 [==============================] - 0s 260us/step - loss: 0.2438 - acc: 0.5500
Epoch 10/50
100/100 [==============================] - 0s 277us/step - loss: 0.2492 - acc: 0.5300
Epoch 11/50
100/100 [==============================] - 0s 254us/step - loss: 0.2390 - acc: 0.6500
Epoch 12/50
100/100 [==============================] - 0s 270us/step - loss: 0.2386 - acc: 0.6000
Epoch 13/50
100/100 [==============================] - 0s 282us/step - loss: 0.2360 - acc: 0.6300
Epoch 14/50
100/100 [================

100/100 [==============================] - 0s 353us/step - loss: 0.2441 - acc: 0.5400
Epoch 6/50
100/100 [==============================] - 0s 279us/step - loss: 0.2337 - acc: 0.6500
Epoch 7/50
100/100 [==============================] - 0s 271us/step - loss: 0.2345 - acc: 0.6400
Epoch 8/50
100/100 [==============================] - 0s 292us/step - loss: 0.2309 - acc: 0.6400
Epoch 9/50
100/100 [==============================] - 0s 227us/step - loss: 0.2305 - acc: 0.5700
Epoch 10/50
100/100 [==============================] - 0s 236us/step - loss: 0.2186 - acc: 0.6100
Epoch 11/50
100/100 [==============================] - 0s 266us/step - loss: 0.2157 - acc: 0.6900
Epoch 12/50
100/100 [==============================] - 0s 270us/step - loss: 0.2058 - acc: 0.6600
Epoch 13/50
100/100 [==============================] - 0s 253us/step - loss: 0.2080 - acc: 0.6700
Epoch 14/50
100/100 [==============================] - 0s 301us/step - loss: 0.2030 - acc: 0.6900
Epoch 15/50
100/100 [===============

100/100 [==============================] - 0s 237us/step - loss: 0.2183 - acc: 0.6500
Epoch 16/50
100/100 [==============================] - 0s 230us/step - loss: 0.2214 - acc: 0.6300
Epoch 17/50
100/100 [==============================] - 0s 243us/step - loss: 0.2310 - acc: 0.6000
Epoch 18/50
100/100 [==============================] - 0s 242us/step - loss: 0.1982 - acc: 0.7400
Epoch 19/50
100/100 [==============================] - 0s 266us/step - loss: 0.2062 - acc: 0.6800
Epoch 20/50
100/100 [==============================] - 0s 270us/step - loss: 0.2080 - acc: 0.6800
Epoch 21/50
100/100 [==============================] - 0s 257us/step - loss: 0.1902 - acc: 0.8300
Epoch 22/50
100/100 [==============================] - 0s 270us/step - loss: 0.1845 - acc: 0.8200
Epoch 23/50
100/100 [==============================] - 0s 300us/step - loss: 0.1691 - acc: 0.8000
Epoch 24/50
100/100 [==============================] - 0s 269us/step - loss: 0.1600 - acc: 0.8400
Epoch 25/50
100/100 [===========

Epoch 8/50
100/100 [==============================] - 0s 219us/step - loss: 0.2283 - acc: 0.6600
Epoch 9/50
100/100 [==============================] - 0s 219us/step - loss: 0.2198 - acc: 0.7300
Epoch 10/50
100/100 [==============================] - 0s 221us/step - loss: 0.2247 - acc: 0.6600
Epoch 11/50
100/100 [==============================] - 0s 228us/step - loss: 0.1999 - acc: 0.8100
Epoch 12/50
100/100 [==============================] - 0s 233us/step - loss: 0.2394 - acc: 0.5300
Epoch 13/50
100/100 [==============================] - 0s 218us/step - loss: 0.2090 - acc: 0.6800
Epoch 14/50
100/100 [==============================] - 0s 219us/step - loss: 0.1985 - acc: 0.7300
Epoch 15/50
100/100 [==============================] - 0s 254us/step - loss: 0.2050 - acc: 0.7200
Epoch 16/50
100/100 [==============================] - 0s 240us/step - loss: 0.1887 - acc: 0.7200
Epoch 17/50
100/100 [==============================] - 0s 225us/step - loss: 0.1860 - acc: 0.7500
Epoch 18/50
100/100 [=

...

100/100 [==============================] - 0s 2ms/step - loss: 0.2519 - acc: 0.4600
Epoch 3/50
100/100 [==============================] - 0s 2ms/step - loss: 0.2496 - acc: 0.5600
Epoch 4/50
100/100 [==============================] - 0s 1ms/step - loss: 0.2480 - acc: 0.5400
Epoch 5/50
100/100 [==============================] - 0s 1ms/step - loss: 0.2482 - acc: 0.5600
Epoch 6/50
100/100 [==============================] - 0s 918us/step - loss: 0.2442 - acc: 0.5400
Epoch 7/50
100/100 [==============================] - 0s 1ms/step - loss: 0.2389 - acc: 0.6100
Epoch 8/50
100/100 [==============================] - 0s 876us/step - loss: 0.2417 - acc: 0.5800
Epoch 9/50
100/100 [==============================] - 0s 969us/step - loss: 0.2360 - acc: 0.6300
Epoch 10/50
100/100 [==============================] - 0s 1ms/step - loss: 0.2401 - acc: 0.5300
Epoch 11/50
100/100 [==============================] - 0s 1ms/step - loss: 0.2394 - acc: 0.5900
Epoch 12/50
100/100 [==============================] 

Epoch 37/50
100/100 [==============================] - 0s 759us/step - loss: 0.1260 - acc: 0.8600
Epoch 38/50
100/100 [==============================] - 0s 538us/step - loss: 0.1296 - acc: 0.8500
Epoch 39/50
100/100 [==============================] - 0s 555us/step - loss: 0.1252 - acc: 0.9000
Epoch 40/50
100/100 [==============================] - 0s 582us/step - loss: 0.1070 - acc: 0.9300
Epoch 41/50
100/100 [==============================] - 0s 818us/step - loss: 0.1030 - acc: 0.9300
Epoch 42/50
100/100 [==============================] - 0s 703us/step - loss: 0.1098 - acc: 0.8900
Epoch 43/50
100/100 [==============================] - 0s 635us/step - loss: 0.1133 - acc: 0.8500
Epoch 44/50
100/100 [==============================] - 0s 763us/step - loss: 0.0924 - acc: 0.9300
Epoch 45/50
100/100 [==============================] - 0s 966us/step - loss: 0.0818 - acc: 0.9500
Epoch 46/50
100/100 [==============================] - 0s 1ms/step - loss: 0.0766 - acc: 0.9600
Epoch 47/50
100/100 [=

100/100 [==============================] - 0s 2ms/step - loss: 0.2335 - acc: 0.6500
Epoch 12/50
100/100 [==============================] - 0s 1ms/step - loss: 0.2259 - acc: 0.6600
Epoch 13/50
100/100 [==============================] - 0s 1ms/step - loss: 0.2214 - acc: 0.7300
Epoch 14/50
100/100 [==============================] - 0s 1ms/step - loss: 0.2194 - acc: 0.7300
Epoch 15/50
100/100 [==============================] - 0s 1ms/step - loss: 0.2090 - acc: 0.6900
Epoch 16/50
100/100 [==============================] - 0s 1ms/step - loss: 0.2046 - acc: 0.7200
Epoch 17/50
100/100 [==============================] - 0s 1ms/step - loss: 0.1829 - acc: 0.7700
Epoch 18/50
100/100 [==============================] - 0s 981us/step - loss: 0.2021 - acc: 0.7100
Epoch 19/50
100/100 [==============================] - 0s 1ms/step - loss: 0.2084 - acc: 0.6800
Epoch 20/50
100/100 [==============================] - 0s 1ms/step - loss: 0.1904 - acc: 0.7600
Epoch 21/50
100/100 [=============================

100/100 [==============================] - 0s 867us/step - loss: 0.1473 - acc: 0.7900
Epoch 34/50
100/100 [==============================] - 0s 1ms/step - loss: 0.1521 - acc: 0.8000
Epoch 35/50
100/100 [==============================] - 0s 965us/step - loss: 0.1318 - acc: 0.8600
Epoch 36/50
100/100 [==============================] - 0s 931us/step - loss: 0.1431 - acc: 0.8200
Epoch 37/50
100/100 [==============================] - 0s 1ms/step - loss: 0.1337 - acc: 0.8200
Epoch 38/50
100/100 [==============================] - 0s 937us/step - loss: 0.1307 - acc: 0.8400
Epoch 39/50
100/100 [==============================] - 0s 891us/step - loss: 0.1519 - acc: 0.7700
Epoch 40/50
100/100 [==============================] - 0s 819us/step - loss: 0.1476 - acc: 0.8000
Epoch 41/50
100/100 [==============================] - 0s 1ms/step - loss: 0.1204 - acc: 0.8800
Epoch 42/50
100/100 [==============================] - 0s 939us/step - loss: 0.1131 - acc: 0.8700
Epoch 43/50
100/100 [=================

100/100 [==============================] - 0s 954us/step - loss: 0.2210 - acc: 0.5900
Epoch 18/50
100/100 [==============================] - 0s 738us/step - loss: 0.2245 - acc: 0.7100
Epoch 19/50
100/100 [==============================] - 0s 1ms/step - loss: 0.2093 - acc: 0.7900
Epoch 20/50
100/100 [==============================] - 0s 1ms/step - loss: 0.2194 - acc: 0.6000
Epoch 21/50
100/100 [==============================] - 0s 994us/step - loss: 0.2042 - acc: 0.7700
Epoch 22/50
100/100 [==============================] - 0s 1ms/step - loss: 0.2026 - acc: 0.7800
Epoch 23/50
100/100 [==============================] - 0s 1ms/step - loss: 0.2005 - acc: 0.7400
Epoch 24/50
100/100 [==============================] - 0s 1ms/step - loss: 0.1933 - acc: 0.7100
Epoch 25/50
100/100 [==============================] - 0s 598us/step - loss: 0.1942 - acc: 0.7400
Epoch 26/50
100/100 [==============================] - 0s 571us/step - loss: 0.1965 - acc: 0.7300
Epoch 27/50
100/100 [=====================